In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
%%time
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

CPU times: total: 0 ns
Wall time: 421 ms


200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)].reset_index(drop='index')
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
0,Зарубежные фильмы,01 Июл 23,Время расплаты / Confidential Informant (2023) WEB-DLRip-AVC от DoMiNo & селезень | P | HDRezka Studio,1.46 GB,47,57
1,Зарубежные фильмы,01 Июл 23,Беглец / Kandahar (2023) WEB-DLRip-AVC от DoMiNo & селезень | D,1.41 GB,38,41
2,Зарубежные фильмы,01 Июл 23,Беглец / Kandahar (2023) WEB-DLRip 720p от DoMiNo & селезень | D,2.60 GB,54,21
3,Зарубежные фильмы,01 Июл 23,Время расплаты / Confidential Informant (2023) WEB-DLRip 720p от DoMiNo & селезень | P | HDRezka Studio,2.74 GB,74,41
4,Зарубежные фильмы,01 Июл 23,Спуск: Новая глава / Dark Nature (2022) WEB-DLRip-AVC от DoMiNo & селезень | GoLTFilm,1.18 GB,96,46
5,Зарубежные фильмы,01 Июл 23,Время расплаты / Confidential Informant (2023) WEB-DL 1080p от New-Team | HDRezka Studio,6.65 GB,361,226
6,Зарубежные фильмы,01 Июл 23,Время расплаты / Confidential Informant (2023) WEB-DLRip от MegaPeer | HDRezka Studio,1.46 GB,491,148
7,Зарубежные фильмы,01 Июл 23,Беглец / Kandahar (2023) WEB-DLRip от ELEKTRI4KA | D,1.46 GB,237,91
8,Зарубежные фильмы,01 Июл 23,"Беглец / Kandahar (2023) WEB-DL 1080p от ELEKTRI4KA | D, P",8.38 GB,185,136
9,Зарубежные фильмы,30 Июн 23,Спуск: Новая глава / Dark Nature (2022) WEB-DL 1080p от New-Team | GoLTFilm,5.18 GB,110,76


In [10]:
id = [21,  ]


my_df.loc[id, 'href']

21    http://rutor.info/torrent/931815/vedmak_the-witcher-03x01-05-iz-08-2023-web-dl-hevc-2160p-ot-newcomers-4k-hdr-dolby-vision-p
Name: href, dtype: object